In [3]:
import pandas as pd
import sqlite3
import pymysql.cursors
from dotenv import load_dotenv
import requests
import zipfile
import shutil
import math

# Load environment variables from the .env file
load_dotenv()
import os

In [11]:
# Connect to the database
connection = pymysql.connect(host=os.getenv("DATABASE_HOST"),
                             user=os.getenv("DATABASE_USERNAME"),
                             password=os.getenv("DATABASE_PASSWORD"),
                             database=os.getenv("DATABASE"),
                             ssl={'ca': os.getenv("CA_CERT_PATH")},
                             cursorclass=pymysql.cursors.DictCursor)

date_range = range(2016, 2022)
tables = ['vpicdecode', 'vehicle']
columns = {"vpicdecode": ['CASENUM', 'VEH_NO', 'VehicleDescriptor', 'VehicleTypeId', 'VehicleType', 'ManufacturerFullNameId', 'ManufacturerFullName', 'MakeId', 'Make', 'ModelId', 'Model', 'PlantCountryId', 'PlantCountry', 'PlantState', 'PlantCity', 'PlantCompanyName', 'BasePrice'],
           "vehicle": ['CASENUM', 'VEH_NO', 'NUMOCCS', 'HIT_RUN', 'VIN', 'MDLYR_IM']}

def add_to_db(table_name, chunk, cols):
    chunk.fillna(-1, inplace=True)
    batch = []
    for index, row in chunk.iterrows():
        t = tuple(row[col_label] for col_label in cols)
        batch.append(t)
    q = 'REPLACE INTO ' + table_name + ' (' + ", ".join(cols) + ') values (' + '%s,' * (len(cols) - 1) + '%s)'
    cursor.executemany(q, batch)

def parse_files_and_add_to_db(table_name, filename, columns):
    csv_path = ""
    path_options = ['%s/%s.csv' % (filename, table_name),
                    '%s/%s.CSV' % (filename, table_name.upper()),
                    '%s/%s.csv' % (filename, table_name.upper()),
                    '%s/%s/%s.csv' % (filename, filename, table_name)]
    for path in path_options:
        if os.path.isfile(path):
            csv_path = path

    chunk_generator = pd.read_csv(csv_path, chunksize=1000, encoding="ISO-8859-1", usecols=columns)

    for chunk in chunk_generator:
        add_to_db(table_name, chunk, columns)

with connection:
    with connection.cursor() as cursor:
        # sqlQuery = "CREATE TABLE Vehicles(Case_Num varchar(32), VEH_Num int, Occupation_Num int, Hit_Run int, VIN varchar(32), Model_Year varchar(32), Make int, Model int)"
        for d in date_range:
            url = "https://static.nhtsa.gov/nhtsa/downloads/CRSS/%s/CRSS%sCSV.zip" % (d, d)
            response = requests.get(url)
            filename = "CRSS%sCSV" % d
            zip_file = filename + ".zip"
            with open(zip_file, "wb") as f:
                f.write(response.content)

            # Unzip the file
            with zipfile.ZipFile(zip_file, "r") as zip_ref:
                zip_ref.extractall(filename)
            
            for table in tables:
                parse_files_and_add_to_db(table, filename, columns[table])
            connection.commit()

            # Delete the zip file
            shutil.rmtree(filename)
            os.remove(zip_file)


In [ ]:
sqlQuery = "CREATE TABLE Accidents(Case_Num varchar(32), VEH_Num int, Occupation_Num int, Hit_Run int, VIN varchar(32), Model_Year varchar(32), Make int, Model int)"

sqlQuery2 = "CREATE TABLE vpicdecode(CASENUM varchar(31),VEH_NO int,VehicleDescriptor varchar(255),VehicleTypeId int,VehicleType varchar(255),ManufacturerFullNameId int,ManufacturerFullName varchar(255),MakeId int,Make varchar(127),ModelId int,Model varchar(127),PlantCountryId int,PlantCountry varchar(63),PlantState varchar(63),PlantCity varchar(63),PlantCompanyName varchar(63),BasePrice dec, PRIMARY KEY (CASENUM, VEH_NO));"